<a href="https://colab.research.google.com/github/viniciusrpb/cic0269_natural_language_processing/blob/main/lectures/cap13_2_word_embeddings_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install tensorflow-datasets
!pip install keras
!pip install tensorflow
!pip install -U gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Embedding,Activation,SimpleRNN,BatchNormalization,RNN,Flatten,Input,LSTM,Bidirectional
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.callbacks import EarlyStopping
from keras.datasets import reuters
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import gensim
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Activation,Embedding,SimpleRNN,Dropout
from keras.utils.np_utils import to_categorical

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp -r '/content/drive/MyDrive/embeddings_STANFORD_GLOVE/GoogleNews-vectors-negative300.bin' 'GoogleNews-vectors-negative300.bin'

In [5]:
ds_train = tfds.load('snli', split='train[50%:]', shuffle_files=True)
ds_valid = tfds.load('snli', split='validation', shuffle_files=False)
ds_test = tfds.load('snli', split='test', shuffle_files=False)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/snli/1.1.0.incompleteBMYK7G/snli-validation.tfrecord*...:   0%|          | 0/1…

Generating train examples...:   0%|          | 0/550152 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/snli/1.1.0.incompleteBMYK7G/snli-train.tfrecord*...:   0%|          | 0/550152…

Dataset snli downloaded and prepared to ~/tensorflow_datasets/snli/1.1.0. Subsequent calls will reuse this data.


In [6]:
df_train = tfds.as_dataframe(ds_train)
df_valid = tfds.as_dataframe(ds_valid)
df_test = tfds.as_dataframe(ds_test)

In [7]:
def preprocessDataFrame(df):

    dic = {}
    dic['premise_hypothesis'] = []
    dic['label'] = []

    hypothesis = [x.decode('utf-8') for x in df['hypothesis'].values]
    premise = [x.decode('utf-8') for x in df['premise'].values]

    for idx,sentence in enumerate(premise):
        dic['premise_hypothesis'].append(premise[idx]+" "+hypothesis[idx])
        dic['label'].append(df['label'][idx])
        
    return pd.DataFrame.from_dict(dic)

In [8]:
df_train = preprocessDataFrame(df_train)
df_valid = preprocessDataFrame(df_valid)
df_test = preprocessDataFrame(df_test)

In [9]:
df_train['label'] = pd.Categorical(df_train['label'])
y_train_int = df_train['label'].cat.codes

df_valid['label'] = pd.Categorical(df_valid['label'])
y_valid_int = df_valid['label'].cat.codes

df_test['label'] = pd.Categorical(df_test['label'])
y_test_int = df_test['label'].cat.codes

y_train = to_categorical(y_train_int)
y_valid = to_categorical(y_valid_int)
y_test = to_categorical(y_test_int)

In [10]:
train_sentences = df_train['premise_hypothesis'].to_list()

vocabulary = {}

for i in range(0,len(train_sentences)):
    train_sentences[i] = train_sentences[i].lower()
    for word in train_sentences[i].split():
        if word not in vocabulary:
            vocabulary[word] = 1
        else:
            vocabulary[word]+=1

In [11]:
tokenizer = Tokenizer(num_words = len(vocabulary))
tokenizer.fit_on_texts(df_train['premise_hypothesis'])
word2index = tokenizer.word_index
vocab_size = len(word2index)

train_sequences = tokenizer.texts_to_sequences(df_train['premise_hypothesis'])
valid_sequences = tokenizer.texts_to_sequences(df_valid['premise_hypothesis'])
test_sequences = tokenizer.texts_to_sequences(df_test['premise_hypothesis'])

In [12]:
trunc_type = 'post'
padding_type = 'post'
max_length = 64
vocab_size = len(vocabulary)

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
valid_padded = pad_sequences(valid_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [13]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=[["hello", "world","hi","earth","sunshine","law"]], vector_size=5, window=5, min_count=1, workers=4)
#model.train([["hi", "earth"]], total_examples=1, epochs=1)

In [14]:
word_vectors = model.wv

In [15]:
len(word_vectors[0])

5

In [16]:
word_vectors['world'] - word_vectors['hello']

array([0.21434264, 0.12033329, 0.01978239, 0.1656901 , 0.20573989],
      dtype=float32)

In [20]:
from gensim.models import KeyedVectors

word_vectors= KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin',
                                                binary=True,
                                                limit=15000)

In [21]:
word_vectors.(positive=['airplane','flight'],negative=['ship'],topn=10) 

[('plane', 0.6277297735214233),
 ('jet', 0.5784463882446289),
 ('flights', 0.5631440877914429),
 ('aircraft', 0.5546182990074158),
 ('airplanes', 0.5451778173446655),
 ('Flight', 0.5407993197441101),
 ('airline', 0.5332231521606445),
 ('planes', 0.5185763835906982),
 ('airport', 0.5175444483757019),
 ('aviation', 0.4899478554725647)]

In [22]:
word_vectors.similarity('orange','apple')

0.39203462

In [23]:
word_vectors.similarity('orange','blue')

0.6421891

In [57]:
embedding_dim = len(word_vectors[0])
vocabulary_size=min(len(word2index)+1,vocab_size)

In [58]:
embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
for word, i in word2index.items():
    if i >= vocab_size:
        continue

    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i] = np.random.normal(0,1,embedding_dim)

In [59]:
from keras.layers import Embedding
from tensorflow import keras
embedding_layer = Embedding(vocabulary_size,
                            embedding_dim,
                            embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                            input_length=max_length,
                            trainable=False)

In [60]:
model = Sequential()
model.add(embedding_layer)
model.add(SimpleRNN(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4,activation="softmax"))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 64, 300)           8372700   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                23360     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                                 
Total params: 8,396,320
Trainable params: 23,620
Non-trainable params: 8,372,700
_________________________________________________________________


In [ ]:
sgd = SGD(learning_rate=0.01)
model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=['accuracy'])
history = model.fit(train_padded,y_train,validation_data=(valid_padded,y_valid),batch_size=64,epochs=30)

Epoch 1/30
4299/4299 [==============================] - 83s 19ms/step - loss: 1.1139 - accuracy: 0.3332 - val_loss: 1.1984 - val_accuracy: 0.3329
Epoch 2/30
 180/4299 [>.............................] - ETA: 1:15 - loss: 1.1104 - accuracy: 0.3356

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
y_prob = model.predict(test_padded)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int,y_pred))